In [ ]:
%pip install langchain langchain-huggingface langchain-pinecone pinecone-notebooks

In [ ]:
%pip install pinecone

In [1]:
from pinecone import Pinecone, ServerlessSpec
import getpass
import os
import time

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [2]:
from langchain.vectorstores import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
# Step 1: Initialize Hugging Face Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Replace with your desired model
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [4]:
# Step 2: Load PDF and Split into Chunks
loader = PyPDFLoader("../../00-example_data/layout-parser-paper.pdf")
pdf_docs = loader.load()

print(f"Loaded {len(pdf_docs)} documents from the file.")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(pdf_docs)

Loaded 16 documents from the file.


In [5]:
import time

index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [6]:
pc.describe_index(index_name)

{
    "name": "langchain-test-index",
    "dimension": 384,
    "metric": "cosine",
    "host": "langchain-test-index-zhmtpmp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

In [8]:
vector_store.add_documents(documents=documents)

['537a6b50-527a-4251-937e-38ede08efd54',
 '3300b4e3-45c9-4660-9e7e-44eb1666c151',
 '0ea4c68e-9b9c-4283-aa42-1dbb4765d5cd',
 '409b8897-1098-4bd5-a878-4d1a83f1cae5',
 '2a39b667-b429-45aa-829f-e9228db72cff',
 '4cfd5d54-366b-4a4b-a079-6ae2bcead69b',
 'e2a0958d-5018-4c4f-9bc1-851e52708236',
 '52e67832-28d9-4e00-80af-5cb8730a482e',
 'cde158e5-84a3-4960-976e-fe60bfd2b5d9',
 '8d4ff1d8-ed7e-4627-ae2f-bc1613b10642',
 'ee9441a8-0bc2-4edd-845e-831a69680652',
 'bda58013-052e-44b7-90ff-f2edb9c39edb',
 'dbe3271b-9e81-4a73-9673-f494039c9fb4',
 '1ef6ee6b-017d-442b-af54-541b9506f071',
 '7332bda0-9639-4a4d-8e2b-ce87942ca576',
 'a31e1b23-8924-4cbd-8db1-b70788eae783',
 'b64d5d06-ab18-4264-9fdc-f07371f1b2fe',
 'bb9d0555-7341-4126-b393-b4880f49c413',
 'd25ef104-c677-4434-8ce6-38b8a58f1242',
 'efc6d6a2-7c5a-4dd2-b6a0-48c6c8ffd961',
 'c53a2753-3375-4536-b98a-66d9815a12cd',
 'b20cce72-0689-4841-8596-fcfec5f7b942',
 '1b319213-fa84-40ee-89b2-864faf5d109b',
 'a56441af-b863-4069-9df1-88d7f66d9ae7',
 '45a8b349-ccef-

In [9]:
pc.list_indexes()

[
    {
        "name": "langchain-test-index",
        "dimension": 384,
        "metric": "cosine",
        "host": "langchain-test-index-zhmtpmp.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    },
    {
        "name": "quickstart",
        "dimension": 2,
        "metric": "cosine",
        "host": "quickstart-zhmtpmp.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

In [16]:

# Step 5: Perform a Query
query = "what is use case being discussed in the documents?"
results = vector_store.similarity_search(query, k=3)

print("\nMost Similar Documents:")
for idx, result in enumerate(results, start=1):
    print(f"{idx}. {result.page_content}")



Most Similar Documents:
1. ument digitization pipelines. For example, sometimes the pipeline requires the
combination of multiple DL models to achieve better accuracy. Currently, pipelines
are mainly described in academic papers and implementations are often not pub-
licly available. To this end, the LayoutParser community platform also enables
the sharing of layout pipelines to promote the discussion and reuse of techniques.
For each shared pipeline, it has a dedicated project page, with links to the source
code, documentation, and an outline of the approaches. A discussion panel is
provided for exchanging ideas. Combined with the core LayoutParser library,
users can easily build reusable components based on the shared pipelines and
apply them to solve their unique problems.
5 Use Cases
The core objective of LayoutParser is to make it easier to create both large-scale
and light-weight document digitization pipelines. Large-scale document processing
2. AllenNLP [8] and transformers [ 

In [17]:
results = vector_store.similarity_search_with_score(
    "what is use case being discussed in the documents?", k=3
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.289667] ument digitization pipelines. For example, sometimes the pipeline requires the
combination of multiple DL models to achieve better accuracy. Currently, pipelines
are mainly described in academic papers and implementations are often not pub-
licly available. To this end, the LayoutParser community platform also enables
the sharing of layout pipelines to promote the discussion and reuse of techniques.
For each shared pipeline, it has a dedicated project page, with links to the source
code, documentation, and an outline of the approaches. A discussion panel is
provided for exchanging ideas. Combined with the core LayoutParser library,
users can easily build reusable components based on the shared pipelines and
apply them to solve their unique problems.
5 Use Cases
The core objective of LayoutParser is to make it easier to create both large-scale
and light-weight document digitization pipelines. Large-scale document processing [{'page': 9.0, 'source': '../../00-example_dat